In [1]:
import numpy as np
import os
import tensorflow as tf
from sklearn.metrics import classification_report ## 印出可視覺化的表

###### Do not modify here ###### 

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

reset_graph()

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

# training on MNIST but only on digits 0 to 4
X_train1 = mnist.train.images[mnist.train.labels < 5]
y_train1 = mnist.train.labels[mnist.train.labels < 5]
X_valid1 = mnist.validation.images[mnist.validation.labels < 5]
y_valid1 = mnist.validation.labels[mnist.validation.labels < 5]
X_test1 = mnist.test.images[mnist.test.labels < 5]
y_test1 = mnist.test.labels[mnist.test.labels < 5]

###### Do not modify here ###### 

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [2]:
##############################################
#                                            #
#               基礎版本                      #
#                                            #
##############################################

In [3]:
### 先前設置 
in_units = 784 # 28 * 28 拉直後 = 784維 
h_units = [128, 128, 128, 128, 128] # 題目要求5層hidden layers，及每一層128neurons
n_class = 5 # 題目要求只要辨識 0 ,1 ,2 ,3 及4 ，共5個類別

n_train = len(X_train1) # train資料的長度
batch_size = 100
n_batch = n_train // batch_size

X = tf.placeholder(tf.float32,[None,in_units], name="X") # 初始化x資料型態為[None,784]
y = tf.placeholder(tf.int64, shape=(None), name="y") # 初始化y資料型態[None]

In [4]:
# [128, 128, 128, 128, 128]
def L_layers_model(X, h_units, n_class, dropout=None):
    # default he_init: factor=2.0, mode='FAN_IN', uniform=False, seed=None, dtype=tf.float32
    he_init = tf.contrib.layers.variance_scaling_initializer()
#     hidden = { "layer0": X }
    
    with tf.name_scope("DNN"):
        hidden = X
        for i in range(len(h_units)):
            hidden = (tf.layers.dense(hidden, h_units[i], 
                                      name="hidden" + str(i+1), 
                                      use_bias=True, kernel_initializer= he_init,
                                      bias_initializer=he_init, activation=tf.nn.elu))
            if dropout:
                hidden = tf.layers.dropout(hidden, rate=dropout)
            
        # 結合之後的 tf.nn.sparse_softmax_cross_entropy_with_logits [128 , 5]
        logits = tf.layers.dense(hidden, n_class, name="logits")
        Y_proba = tf.nn.softmax(logits, name="Y_proba")
    
    return Y_proba

def train_op(y, logits):
    with tf.name_scope("calc_loss"):
        entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
        loss = tf.reduce_mean(entropy, name="loss")
    
    ## 此區使用AdamOptimizer 的優化器進行梯度優化
    with tf.name_scope("train"):
        batch = tf.Variable(0)

        learning_rate = tf.train.exponential_decay(
                1e-4,  # Base learning rate.
                batch * batch_size,  # Current index into the dataset.
                n_train,  # Decay step.
                0.95,  # Decay rate.
                staircase=True)
        
        optimizer = tf.train.AdamOptimizer(learning_rate)
        training_op = optimizer.minimize(loss, global_step=batch, name="training_op")

    return (loss, training_op)

def acc_model(y, logits):
    #計算正確率   
    with tf.name_scope('calc_accuracy'):
        correct = tf.equal(tf.argmax(logits, 1), y)
        accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

    #計算precision 因返回會有兩個值，只取後者
    with tf.name_scope('calc_precision'):
        _, precision = tf.metrics.precision(predictions=tf.argmax(logits,1), labels=y, name="precision")

    #計算recall 因返回會有兩個值，只取後者
    with tf.name_scope('calc_recall'):
        _, recall = tf.metrics.recall(predictions=tf.argmax(logits,1), labels=y, name="recall")

    return (accuracy, precision, recall)


def shuffle_data(data, labels):
    idx = np.random.permutation(len(data))
    data, label = data[idx], labels[idx]
    return (data, label)

In [5]:
logits = L_layers_model(X, h_units, n_class)
loss, train_op = train_op(y, logits)
accuracy, precision, recall = acc_model(y, logits)

checkpoint_dir = 'save/'  # model儲存位址
saver = tf.train.Saver()  # call save function

config = tf.ConfigProto(device_count = {'GPU': 1}) #指定gpu

In [6]:
# Params for Train
epochs = 1000 # 10 for augmented training data, 20 for training data
val_step = 50 # 當 50 步時去算一次驗證資料的正確率
early_stop = 20
# display_step = 100  # 你可以用這個去建一個規則去看train的正確率，但本專案先不用

init = tf.global_variables_initializer()
init_l = tf.local_variables_initializer()

In [7]:
def run(sess, X_train1, y_train1, X_valid1, y_valid1):
    max_acc = 0. # Save the maximum accuracy value for validation data
    early_stop_limit = 0 # 紀錄early_stop的值
        
    sess.run([init, init_l])
    summary_writer = tf.summary.FileWriter('tensorboard/', graph=tf.get_default_graph())
    
    # 可以自己選擇大概要幾個epoch，因為我們有early stop的方法，所以不怕一直train
    for epoch in range(epochs):
        if early_stop_limit >= early_stop: 
            print('early_stop...........')
            break

        # Random shuffling
        train_data, train_label = shuffle_data(X_train1, y_train1)

        # 用批次的方式去訓練 model
        for i in range(n_batch):
            # Compute the offset of the current minibatch in the data.
            offset = (i * batch_size) % (n_train)
            batch_xs = train_data[offset:(offset + batch_size), :]
            batch_ys = train_label[offset:(offset + batch_size)]
            sess.run([train_op, loss], feed_dict={X: batch_xs, y: batch_ys})

            # 每 n step時，model去看此時的驗證資料的正確率並印出來
            if i % val_step == 0:
                val_acc = sess.run(accuracy, feed_dict={X: X_valid1, y: y_valid1})
                print("Epoch:", '%04d,' % (epoch + 1), 
                      "batch_index %4d/%4d , validation accuracy %.5f" % (i, n_batch, val_acc))

                # 透過最大驗證正確率大於每一次的驗證正確率的條件來設定 early stop
                if max_acc >= val_acc:
                    early_stop_limit += 1
                    if early_stop_limit == early_stop: # 自己可以去限制最大驗證正確率不再變n次時，就停止訓練
                        break

                # 如果 val_acc 大於 max_acc，則取代它並儲存一次結果
                else: # validation_accuracy > max_acc
                    early_stop_limit = 0
                    max_acc = val_acc
                    saver.save(sess, checkpoint_dir + 'Team07_HW2.ckpt')                        
                    print('Team07_HW2.ckpt-' + 'complete-%04d-' % (epoch + 1) + 
                          "batch_index-%d" % i)

In [8]:
with tf.Session(config=config) as sess:
    run(sess, X_train1, y_train1, X_valid1, y_valid1)
    
    sess.run(init_l)
    saver.restore(sess, 'save/Team07_HW2.ckpt') # 開啟剛剛 early_stop 的 model

    print('Acc_test :' , sess.run(accuracy, feed_dict={X: X_test1, y: y_test1}))
    print('Prec_value :' , sess.run(precision, feed_dict={X: X_test1, y: y_test1}))
    print('Recall_value :' , sess.run(recall, feed_dict={X: X_test1, y: y_test1}))
    
    ## model去測試測試集並將label的結果記錄下來 
    pre = sess.run(tf.argmax(logits, 1), feed_dict={X: X_test1})

    ## 列印出測試資料的報表
    print(classification_report(y_test1.ravel(), pre))

Epoch: 0001, batch_index    0/ 280 , validation accuracy 0.27600
Team07_HW2.ckpt-complete-0001-batch_index-0
Epoch: 0001, batch_index   50/ 280 , validation accuracy 0.92377
Team07_HW2.ckpt-complete-0001-batch_index-50
Epoch: 0001, batch_index  100/ 280 , validation accuracy 0.94801
Team07_HW2.ckpt-complete-0001-batch_index-100
Epoch: 0001, batch_index  150/ 280 , validation accuracy 0.95622
Team07_HW2.ckpt-complete-0001-batch_index-150
Epoch: 0001, batch_index  200/ 280 , validation accuracy 0.96013
Team07_HW2.ckpt-complete-0001-batch_index-200
Epoch: 0001, batch_index  250/ 280 , validation accuracy 0.96325
Team07_HW2.ckpt-complete-0001-batch_index-250
Epoch: 0002, batch_index    0/ 280 , validation accuracy 0.96560
Team07_HW2.ckpt-complete-0002-batch_index-0
Epoch: 0002, batch_index   50/ 280 , validation accuracy 0.96716
Team07_HW2.ckpt-complete-0002-batch_index-50
Epoch: 0002, batch_index  100/ 280 , validation accuracy 0.96833
Team07_HW2.ckpt-complete-0002-batch_index-100
Epoch: 

In [9]:
###################################
#                                 #
#     K-FOLD cross validation     #
#                                 #
###################################

In [10]:
### 額外程式，主要去打亂合併的資料，並做k-fold 的取資料
class CrossValidationFolds(object):
    
    def __init__(self, data, labels, num_folds, shuffle=True):
        self.data = data
        self.labels = labels
        self.num_folds = num_folds
        self.current_fold = 0
        
        # Shuffle Dataset
        if shuffle:
            perm = np.random.permutation(self.data.shape[0]) ##隨機打亂資料
            data = data[perm]
            labels = labels[perm]
    
    def split(self):
        current = self.current_fold
        size = int(self.data.shape[0]/self.num_folds) # 30596 / 5 一塊k的size大小
        
        index = np.arange(self.data.shape[0]) 

        # 利用 True/False 抓出 validation 區塊
        lower_bound = index >= current*size # validation 下界
        upper_bound = index < (current + 1)*size # 上界

        cv_region = lower_bound*upper_bound

        cv_data = self.data[cv_region] # 利用 True/False 抓出 True 的資料
        train_data = self.data[~cv_region]
        
        cv_labels = self.labels[cv_region]
        train_labels = self.labels[~cv_region]
        
        self.current_fold += 1 ## 丟回下一的fold
        return (train_data, train_labels), (cv_data, cv_labels)

In [11]:
FOLDS = 5 # k = 5

X_train = np.concatenate((X_train1,X_valid1), axis=0)
Y_label = np.concatenate((y_train1,y_valid1), axis=0)
data = CrossValidationFolds(X_train, Y_label, FOLDS)

recall_test = [] # 放每個第n-fold 時的recall值
prec_test = []   # 放每個第n-fold 時的精確值
acc_test = []    # 放每個第n-fold 時的正確率值

In [12]:
with tf.Session(config=config) as sess:
    for i in range(FOLDS):
        print("Fold: ", '%d,' % (i+1))
        (X_train1, y_train1), (X_valid1, y_valid1) = data.split()
        run(sess, X_train1, y_train1, X_valid1, y_valid1)
                        
        recall_test.append(sess.run(accuracy, feed_dict={X: X_test1, y: y_test1}))
        prec_test.append(sess.run(precision, feed_dict={X: X_test1, y: y_test1}))
        acc_test.append(sess.run(recall, feed_dict={X: X_test1, y: y_test1}))
    
    print(acc_test)
    print(recall_test)
    print(prec_test)
    print('1. 5-Fold平均正確率 : ',np.array(acc_test).mean() , 
          '2. 5-Fold平均精確率 : ',np.array(prec_test).mean() ,
          '3. 5-Fold平均Recall : ',np.array(recall_test).mean())

Fold:  1,
Epoch: 0001, batch_index    0/ 280 , validation accuracy 0.27292
Team07_HW2.ckpt-complete-0001-batch_index-0
Epoch: 0001, batch_index   50/ 280 , validation accuracy 0.91404
Team07_HW2.ckpt-complete-0001-batch_index-50
Epoch: 0001, batch_index  100/ 280 , validation accuracy 0.94068
Team07_HW2.ckpt-complete-0001-batch_index-100
Epoch: 0001, batch_index  150/ 280 , validation accuracy 0.94917
Team07_HW2.ckpt-complete-0001-batch_index-150
Epoch: 0001, batch_index  200/ 280 , validation accuracy 0.95440
Team07_HW2.ckpt-complete-0001-batch_index-200
Epoch: 0001, batch_index  250/ 280 , validation accuracy 0.95620
Team07_HW2.ckpt-complete-0001-batch_index-250
Epoch: 0002, batch_index    0/ 280 , validation accuracy 0.95506
Epoch: 0002, batch_index   50/ 280 , validation accuracy 0.95931
Team07_HW2.ckpt-complete-0002-batch_index-50
Epoch: 0002, batch_index  100/ 280 , validation accuracy 0.96225
Team07_HW2.ckpt-complete-0002-batch_index-100
Epoch: 0002, batch_index  150/ 280 , vali